# Teste - Técnicas de Análise Sintática


* **nome:** Aluno(a) 1
* **matrícula:** 12/345678
* **nome:** Aluno(a) 2
* **matrícula:** 98/765432

In [276]:
# Import o lark
try:
    import lark
except ImportError:
    !pip install lark-parser --user
    import lark

## Questão 1 - Descida recursiva

### Introdução

A gramática abaixo descreve de forma simplificada uma parte da sintaxe do SQL na notação do Lark. Neste exercício implementaremos um analisador sintático de descida recursiva para esta gramática.

In [318]:
grammar = r"""
// Produções
start   : select

select  : "SELECT" columns "FROM" name [where] [orderby] ";"

columns : "*"   -> columns_all
        | names

names   : name ("," name)*

where   : "WHERE" condition

condition : name op value

orderby : "ORDER" "BY" names direction


// Terminais
name  : NAME
op    : OP
value : VALUE | NAME
direction : DIRECTION


// Expressões regulares
DIRECTION.2 : /ASC|DESC/
NAME.1  : /(?!\d)\w+/
VALUE.0 : /\w+/
OP    : /=|<|>|>=|<=/


%ignore /\s+/
"""

No exercício, utilizaremos o Lark apenas para fazer a análise léxica e para conferir se a resposta está correta. Para isso, vamos definir algumas funções

In [278]:
parser = lark.Lark(grammar, parser='lalr')

# Analisador léxico
lex = lambda st: list(parser.lex(st))

# Imprime o resultado na forma "pretty print"
pretty = lambda st: print(parser.parse(st).pretty())

# Transforma a árvore sintática em dicionários e tipos Python básicos
to_expr = lambda st: DictTransformer().transform(parser.parse(st))

Usamos o transformer abaixo para converter a árvore sintática para um dicionário. Usamos este código apenas como referência de como deve ser o resultado esperado da árvore resultante da análise sintática

In [279]:
class DictTransformer(lark.InlineTransformer):
    """
    Converte árvore para dicionário
    """
    start = lambda _, ast: ast 
    name = op = value = direction = str
    condition = lambda _, a, op, b: [op, a, b]
    where = lambda _, cond: {'where': cond}
    orderby = lambda _, names, direction: {'order_by': [direction, *names]}
    names = lambda _, *args: list(args)
    columns = lambda _, names: names
    columns_all = lambda _: None
    
    def select(self, columns, table, *args):
        result = {
            'command': 'select', 
            'columns': columns, 
            'table': table,
        }
        for arg in args:
            result.update(arg)
        return result

### Exemplos

Considere alguns exemplos abaixo e os resultados esperados tanto na forma "pretty print" quanto na forma final a ser implementada com a descida recursiva. 

In [284]:
# Exemplos
bad_1 = 'SELECT * FROM users'  # sem ponto-vírgula no final
bad_2 = 'SELECT foo,bar WHERE age = 0;'  # sem a cláusula FROM
bad_3 = 'SELECT foo,bar FROM users ORDER BY age;'  # sem ordenamento ASC/DESC
bad_4 = 'SELECT * FROM users ASC;' # ASC sem ORDER BY 
bad_5 = 'SELECT * FROM users; SELECT' # Valores adicionais

cmd_simple = 'SELECT * FROM users;'
cmd_simple_2 = 'SELECT col1,col2 FROM users;'
cmd_ord = 'SELECT * FROM users ORDER BY name ASC;'
cmd_where = 'SELECT * FROM users WHERE name = surname;'
cmd_full = 'SELECT name, age FROM users WHERE age > 18 ORDER BY name ASC;'

good = [cmd_simple, cmd_simple_2, cmd_ord, cmd_where, cmd_full]
bad = [bad_1, bad_2, bad_3, bad_4, bad_5]

cmd = cmd_full

Veja o resultado esperado

In [285]:
# Pretty printing
pretty(cmd)

start
  select
    columns
      names
        name	name
        name	age
    name	users
    where
      condition
        name	age
        op	>
        value	18
    orderby
      names
        name	name
      direction	ASC



In [286]:
# Seu parser deve emitir uma saída como esta
to_expr(cmd)

{'columns': ['name', 'age'],
 'command': 'select',
 'order_by': ['ASC', 'name'],
 'table': 'users',
 'where': ['>', 'age', '18']}

### Uma pausa para metaclasses...

Abaixo temos uma implementação da metaclasse do nosso Parser. Não tente entender isto agora, pule para a próxima seção!

In [192]:
from collections import deque
from functools import wraps

class RecursiveParserMeta(type):
    """
    Metaclasse* para criar um parser de descida recursiva. 
    Implementa um sistema de debug automático
    
    * metaclasse = "hoje é dia de maldade!"
    """
    def __new__(cls, name, bases, ns):
        def decorator(name, func):
            @wraps(func)
            def method(self, *args, **kwargs):
                try:
                    self._depth += 1
                    prefix = (self._depth - 1) * 4 * ' '
                    print('%scalled %s()' % (prefix, name))
                    res = func(self, *args, **kwargs)
                    self._depth -= 1
                    print('%s%s() -> %r' % (prefix, name, res))
                    if self.debug >= 2:
                        tk_show = ('%s(%r)' % (tk.type, str(tk)) for tk in self.tokens)
                        print('%s  - tokens: %s\n' % (prefix, ' '.join(tk_show)))
                    return res
                except Exception as exc:
                    print('error calling %s(): %s' % (name, exc))
                    raise
            return method
        
        if ns.get('debug'):
            for k, v in ns.items():
                if callable(v):
                    ns[k] = decorator(k, v)
        return super().__new__(cls, name, bases, ns)

### Parser de descida recursiva

Agora temos o parser da descida recursiva. Nossa classe base implementa alguns métodos úteis como pop(), peek() e is_empty(). Leia o código abaixo para se familizarizar com a função de cada método.

In [324]:
class RecursiveParser(metaclass=RecursiveParserMeta):
    """
    Implementação base do parser recursivo
    """
    check_finished = True
    _depth = 0
    
    @classmethod
    def parse(cls, src):
        """
        Executa parser para uma string de código de entrada. Executada como método da classe.
        
        >>> MyParserClass.parse("some string of code")
        (...)
        """
        parser = cls(lex(src))
        ast = parser.start()
        if parser.is_empty() or not parser.check_finished:
            return ast
        raise SyntaxError('parsing finished with unread tokens: %s' % parser.peek())
    
    def __init__(self, tokens):
        self.tokens = deque(tokens)
                
    def is_empty(self):
        "Retorna True se a lista de tokens estiver exaurida"
        return not bool(self.tokens)
        
    def peek(self):
        "Mostra próximo token ou None"
        return self.tokens[0] if self.tokens else None
    
    def peek_type(self):
        "Mostra o tipo do próximo token ou None"
        return self.tokens[0].type if self.tokens else None
    
    def pop(self, kind=None):
        "Consome o próximo token, retornando-o."
        if not self.tokens:
            if kind:
                raise SyntaxError('EOF when trying to read a %s' % kind)
            return None
        
        tk = self.tokens.popleft()
        if kind and tk.type != kind:
            raise SyntaxError('Expected a %r, but got a %r' % (kind, tk.type))
        return tk
    
    def push(self, tk):
        "Devolve token para o início da lista"
        self.tokens.appendleft(tk)
        
    def start(self):
        raise NotImplementedError('você deve implementar o método start em uma sub-classe')

### SqlParser

Agora vamos implementar nosso parser herdando as funcionalidades genéricas do RecusiveParser. A parte mais importante desta classe é sobrescrever o método start(). É lógico que para implementar a descida recursiva, 
devemos implementar vários outros métodos chamados a partir deste método inicial. 

A classe abaixo fornece um esqueleto da implementação com um método para cada regra sintática da gramática. Você deve completar a implementação e pode simplificar algumas regras ou métodos opcionalmente.

In [301]:
class SqlParser(RecursiveParser):
    check_finished = False  # Se False, retorna árvore sintática mesmo se ainda existirem tokens não-lidos
    debug = 2  # 0, 1, ou 2, para mensagens de erro
    
    def start(self):
        """
        start   : select
        """
        return NotImplemented
    
    def select(self):
        """
        select  : "SELECT" columns "FROM" name [where] [orderby] ";"
        """
        columns = [...]
        table = [...]
        where = {}
        orderby = {}
        return {
            "command": "select",
            "columns": columns,
            "table": table,
            **where,
            **orderby,
        }

    def columns(self):
        """
        columns : "*"   -> columns_all 
                | names
        """
        return None # p/ * ou uma tupla para lista de nomes
    
    def names(self):
        """
        names   : name ("," name)*
        """
        return [...]
    
    def where(self):
        """
        where   : "WHERE" condition
        """
        condition = ...
        return {'where': condition}
    
    def condition(self):
        """
        condition : name op value
        """
        name = ...
        op = ...
        value = ...
        return [op, name, value]
    
    def orderby(self):
        """
        orderby : "ORDER" "BY" names direction
        """
        names = ...
        direction = ...
        return {"order_by": [direction, *names]}
    
    # Terminais (com alguns pontos de bônus)
    name = lambda self: str(self.pop('NAME'))
    op = lambda self: str(self.pop('OP'))
    direction = lambda self: str(self.pop('DIRECTION'))
    
    def value(self):
        """
        value : NAME | VALUE
        """
        return ...

### Testando o resultado

Chamamos o parser com o método de classe .parse()

O nível de debug pode ser controlado com as variáveis de classe "debug" e "check_finished". No exemplo, deixamos o debug no máximo e check_finished = False. Uma vez que o código estiver pronto e funcionando, mude debug=0 e check_finished=True.

In [302]:
SqlParser.parse(cmd)

called start()
start() -> NotImplemented
  - tokens: SELECT('SELECT') NAME('name') COMMA(',') NAME('age') FROM('FROM') NAME('users') WHERE('WHERE') NAME('age') OP('>') VALUE('18') ORDER('ORDER') BY('BY') NAME('name') DIRECTION('ASC') SEMICOLON(';')



NotImplemented

### Testes unitários

In [328]:
# Exemplos positivos
from pprint import pprint

for cmd in good:
    try:
        ast_got = SqlParser.parse(cmd)
    except SyntaxError as exc:
        print('Erro de sintaxe inválida no comando:\n  %s' % cmd)
        print(exc, '\n')
        continue
    ast_expected = to_expr(cmd)
    
    if ast_got != ast_expected:
        print('Resultado incorreto no comando:\n  %s' % cmd)
        print('Experado:')
        pprint(ast_expected)
        print('Obtido:')
        pprint(ast_got)
print('Testes finalizados!')

Testes finalizados!


In [330]:
# Testes negativos

for cmd in bad:
    try:
        SqlParser.parse(cmd)
    except SyntaxError:
        pass
    except Exception as exc:
        print('Erro de sintaxe inválida no comando:\n  %s' % cmd)
        print(exc, '\n')
    else:
        print('Aceitou comando inválido:\n  %s' % cmd)

## Questão 2 - Parser LL(1)

Na segunda questão, vamos implementar um parser LL(1) para uma gramática simples de uma calculadora. A gramática de referência na notação do Lark segue abaixo. Observe que como o Lark não aceita produções vazias, tivemos que usar um pouco de criatividade para exprimir a gramática.

In [333]:
grammar = r"""
start    : expr

expr     : term expr_rhs

expr_rhs : ["+" term expr_rhs]
      // | epsilon

term     : atom term_rhs

term_rhs : ["*" atom term_rhs]
      // | epsilon

atom     : "(" expr ")"
         | INT

INT      : /\d+/
%ignore /\s+/
"""

Utilizamos o Lark para criar o lexer e uma implementação de referência.

In [341]:
from lark import Token

EOF = Token('EOF', '$')
parser = lark.Lark(grammar, parser='lalr')
lex = lambda st: [*parser.lex(st), EOF]

Veja os exemplos

In [342]:
print('PARSING\n')
print(parser.parse('(20 + 1) * 2').pretty())

print('\nLEXER')
display(lex('(20 + 1) * 2'))

PARSING

start
  expr
    term
      atom
        expr
          term
            atom	20
            term_rhs
          expr_rhs
            term
              atom	1
              term_rhs
            expr_rhs
      term_rhs
        atom	2
        term_rhs
    expr_rhs


LEXER


[Token(LPAR, '('),
 Token(INT, '20'),
 Token(PLUS, '+'),
 Token(INT, '1'),
 Token(RPAR, ')'),
 Token(STAR, '*'),
 Token(INT, '2'),
 Token(EOF, '$')]

### Representação da gramática como estrutura de dados

Vamos representar a gramática como uma estrutura de dados. Para isto, usamos um dicionário de strings representando os símbolos terminais para uma lista de produções para este terminal. Cada produção, por sua vez, é uma lista de símbolos. 

Representamos os símbolos não terminais por strings e os terminais como funções que lêem um token e retornam True, se o token for do tipo esperado, e False caso contrário.

In [343]:
def tk(name): 
    """
    tk(name) -> cria uma função que representa uma categoria de tokens.
    
    A função resultante recebe um valor e retorna se o mesmo é um token do tipo especificado.
    
    Ex:
    >>> fn = tk('NAME')
    >>> fn(lark.Token('NAME', 'hi')), fn(lark.Token('PLUS', '+')), fn('non-terminal')
    True, False, False
    """ 
    func = lambda tk: getattr(tk, 'type', None) == name
    func.__name__ = name
    func.__qualname__ = name
    return func

grammar = {
    'expr': [
        ['term', 'expr_rhs'],
    ],
    'expr_rhs': [
        [tk('PLUS'), 'term', 'expr_rhs'],
        [], # epsilon
    ],
    'term': [
        ['atom', 'term_rhs'],
    ],     
    'term_rhs': [
        [tk('STAR'), 'atom', 'term_rhs'],
        [], # epsilon
    ],
    'atom':[
        [tk('LPAR'), 'expr', tk('RPAR')],
        [tk('INT')],
    ],
}

Neste exercício, já fornecemos a tabela de parsing para vocês como um dicionário de (não-terminal, terminal) -> regra de produção.

In [ ]:
expr = grammar['expr']
expr_rhs = grammar['expr_rhs']
term = grammar['term']
term_rhs = grammar['term_rhs']
atom = grammar['atom']

parse_table = {
    # expr
    ('expr', 'LPAR'): expr[0],
    ('expr', 'INT'): expr[0],
    
    # expr_rhs
    ('expr_rhs', 'PLUS'): expr_rhs[0],
    ('expr_rhs', 'RPAR'): expr_rhs[1],
    ('expr_rhs', 'EOF'):  expr_rhs[1],
    
    # term
    ('term', 'LPAR'): term[0],
    ('term', 'INT'):  term[0],
    
    # term_rhs
    ('term_rhs', 'STAR'): term_rhs[0],
    ('term_rhs', 'PLUS'): term_rhs[1],
    ('term_rhs', 'RPAR'): term_rhs[1],
    ('term_rhs', 'EOF'):  term_rhs[1],
    
    # atom
    ('atom', 'LPAR'): atom[0],
    ('atom', 'INT'):  atom[1],
}

### Parser LL(1), detectando valores válidos

Crie uma função que implemente o algoritmo LL(1) para verificar se uma expressão de entrada fornecida como uma
string faz parte da linguagem considerada ou não.

In [344]:
def LL1_test(src: str) -> bool:
    return False

LL1_test('(20 + 1) * 2')

False

### Parser LL(1), construindo a árvore sintática

Crie uma função que implemente o algoritmo LL(1) e construa a árvore sintática concreta a partir daí. Expresse a árvore concreta como uma s-expression.

Dica: para construir a árvore, devemos manter uma pilha de nós representados como listas. A cada vez que uma substituição for realizada, devemos adicionar um elemento novo na pilha. 

In [347]:
def LL1_parse(src: str) -> bool:
    return False

LL1_parse('40 + 2') == \
['start',
    ['expr',
      ['term', 
           ['atom', Token('INT', '40')], 
           ['term_rhs']],
      ['expr_rhs',
           Token('PLUS', '+'),
           ['term', 
                ['atom', Token('INT', '2')], 
                ['term_rhs']],
           ['expr_rhs']]]]

False

In [256]:


def ll1_test(src):
    tokens = lex(src)
    instructions = ['expr', tk('EOF')]

    while instructions:
        inst = instructions.pop(0)
        if callable(inst):
            if inst(tokens[0]):
                tokens.pop(0)
            else:
                raise SyntaxError('unexpected token: %r' % tokens[0])
        else:
            new = parse_table[inst, tokens[0].type]
            instructions = [*new, *instructions]
    return True

ll1_test('(20 + 1) * 2')

True

In [345]:
def ll1_parse(src):
    tokens = lex(src)
    instructions = ['expr', tk('EOF')]
    root = node = ['start']
    sequence = [node]
    
    while instructions:
        instr = instructions.pop(0)
        if callable(instr):
            if instr(tokens[0]):
                node.append(tokens.pop(0))
            else:
                raise SyntaxError('unexpected token: %r' % tokens[0])
        elif instr == ...:
            node = sequence.pop()
        else:
            child = [instr]
            node.append(child)
            sequence.append(node)
            node = child
            extra = parse_table[instr, tokens[0].type]
            instructions = [*extra, ..., *instructions]
    return root

ll1_parse('40 + 2')

['start',
 ['expr',
  ['term', ['atom', Token(INT, '40')], ['term_rhs']],
  ['expr_rhs',
   Token(PLUS, '+'),
   ['term', ['atom', Token(INT, '2')], ['term_rhs']],
   ['expr_rhs']]],
 Token(EOF, '$')]

In [327]:
class SqlParser(RecursiveParser):
    check_finished = False
    debug = 0
    
    def start(self):
        """
        start   : select
        """
        return self.select()
    
    def select(self):
        """
        select  : "SELECT" columns "FROM" name [where] [orderby] ";"
        """
        self.pop('SELECT')
        columns = self.columns()
        self.pop('FROM')
        table = self.name()
        where = self.where() if self.peek_type() == "WHERE" else {}
        orderby = self.orderby() if self.peek_type() == "ORDER" else {}
        self.pop('SEMICOLON')
        return {
            "command": "select",
            "columns": columns,
            "table": table,
            **where,
            **orderby,
        }

    def columns(self):
        """
        columns : "*"   -> columns_all 
                | names
        """
        if self.peek_type() == 'STAR':
            self.pop()
            return None
        return self.names()
    
    def names(self):
        """
        names   : name ("," name)*
        """
        names = [self.name()]
        while self.peek_type() == 'COMMA':
            self.pop()
            names.append(self.name())
        return names
    
    def where(self):
        """
        where   : "WHERE" condition
        """
        self.pop('WHERE')
        condition = self.condition()
        return {'where': condition}
    
    def condition(self):
        """
        condition : name op value
        """
        a = self.name()
        op = self.op()
        b = self.value()
        return [op, a, b]
    
    def orderby(self):
        """
        orderby : "ORDER" "BY" names direction
        """
        self.pop('ORDER')
        self.pop('BY')
        names = self.names()
        direction = self.direction()
        return {"order_by": [direction, *names]}
    
    # Terminais
    def value(self):
        """
        value : NAME | VALUE
        """
        if getattr(self.peek(), 'type', None) == 'VALUE':
            return str(self.pop())
        else:
            return str(self.pop('NAME'))
    
    name = lambda self: str(self.pop('NAME'))
    op = lambda self: str(self.pop('OP'))
    direction = lambda self: str(self.pop('DIRECTION'))
    
    
SqlParser.parse(cmd_full)

{'columns': ['name', 'age'],
 'command': 'select',
 'order_by': ['ASC', 'name'],
 'table': 'users',
 'where': ['>', 'age', '18']}